In [1]:
import pickle
from datetime import datetime
import pandas as pd 
import os

In [2]:
def load_pickle(data_path):
    with open(data_path,'rb') as f:
        data = pickle.load(f)
    return data
def save_pickle(data,save_path):
    with open(save_path,'wb') as f:
        pickle.dump(data,f)

## 1. convert to Recbole and SFT format

### 1. Recoble format
- category.pickle
- dataset_name.inter
- dataset_name.item

### 2. SFT format
- category.pickle 'category': [item_ids]
- metas.pickle 'item_id': {'item_id', 'title'}
- sequential.pickle 'user_id': [item_ids]

In [3]:
file_path = '/home/wangshuo/codes/rec-scope/cbs/train_on_output/dataset/steam/base/steam_final.pickle'
final_df = load_pickle(file_path)
final_df.head()

,user_id,interactions
7503577,7878071,"[(Divinity: Original Sin - Enhanced Edition, 3..."
6563456,6917694,"[(Prison Architect, 233450, 2016-03-01 00:00:0..."
7873593,8256766,"[(The Long Dark, 305620, 2015-03-10 00:00:00, ..."
8656357,9059167,"[(Darkest Dungeon®, 262060, 2015-08-24 00:00:0..."
10574607,11030775,"[(Space Engineers, 244850, 2014-04-10 00:00:00..."


In [ ]:
final_df.iloc[0]['interactions']

In [8]:
sequential = {}
metas ={}
category = {}
iter =[]
item_recbole = []
COMMON_TAG = 'game'
miss_count = 0

for index, row in final_df.iterrows():
    user_id = str(row['user_id'])
    # interactions(item_title,item_id, timestamp, tags(list))
    interactions = row['interactions']
    item_ids = []
    for interaction in interactions:
        item_title = interaction[0].strip()
        item_id = str(interaction[1])
        timestamp = interaction[2]
        tags = interaction[3]
        
        if len(tags) == 0:
            tags = [COMMON_TAG]
            miss_count += 1

        for tag in tags:
            tag =tag.strip()
            if tag not in category:
                category[tag] = []
            if item_id not in category[tag]:
                category[tag].append(item_id)
            # category[tag].append(item_id)
        iter.append({"user_id:token":user_id,"item_id:token":item_id,"timestamp:float":timestamp.timestamp()})
        # item_recbole.append({"item_id:token":item_id,"timestamp:float":toTimestamp(timestamp)})
        item_ids.append(item_id)
        metas[item_id]={"title":item_title}
    
    sequential[user_id]=item_ids
print('miss_count:',miss_count)


miss_count: 66209


In [34]:
iter_df = pd.DataFrame(iter)
iter_df = iter_df[["user_id:token","item_id:token","timestamp:float"]]
iter_df.to_csv("/home/wangshuo/codes/InstructControllableRec_RLHF/SASRec/dataset/steam/steam.inter",index=False,sep='\t')

In [7]:
save_base_dir = "/home/wangshuo/codes/InstructControllableRec_RLHF/data/dataset/steam/"
save_pickle(sequential,os.path.join(save_base_dir,"sequential.pickle"))
save_pickle(metas,os.path.join(save_base_dir,"metas.pickle"))
save_pickle(category,os.path.join(save_base_dir,"category.pickle"))

In [4]:
save_base_dir = "/home/wangshuo/codes/InstructControllableRec_RLHF/data/dataset/steam/"
data = load_pickle(os.path.join(save_base_dir,"metas.pickle"))

In [16]:
len(set(category[COMMON_TAG]))

555